In [1]:
import pandas as pd
import pyreadstat as st
pd.set_option('display.max_columns', None)
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 2T\[MT08-MT13] Microeconometría II\Clases prácticas\PS\PS3-20251111\Panel101.dta"

df, meta = st.read_dta(path)
df.head(1)

,country,year,y,y_bin,x1,x2,x3,opinion,op,Y,time,treated,did,_diff
0,1,1990,1.342788e+09,1.0,0.277904,-1.107956,0.282554,1.0,1.0,1.342788,0.0,0.0,0.0,0.0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  70 non-null     int64  
 1   year     70 non-null     int64  
 2   y        70 non-null     float64
 3   y_bin    70 non-null     float64
 4   x1       70 non-null     float64
 5   x2       70 non-null     float64
 6   x3       70 non-null     float64
 7   opinion  70 non-null     float64
 8   op       70 non-null     float64
 9   Y        70 non-null     float64
 10  time     70 non-null     float64
 11  treated  70 non-null     float64
 12  did      70 non-null     float64
 13  _diff    70 non-null     float64
dtypes: float64(12), int64(2)
memory usage: 7.8 KB


In [3]:
meta.column_names_to_labels

{'country': 'Country',
 'year': 'Year',
 'y': 'Outcome Y',
 'y_bin': 'Binary outcome Y',
 'x1': 'Predictor x1',
 'x2': 'Predictor x2',
 'x3': 'Predictor x3',
 'opinion': 'Categorical variable',
 'op': None,
 'Y': None,
 'time': None,
 'treated': None,
 'did': None,
 '_diff': 'Diff-in-diff'}

In [5]:
df.describe()

,country,year,y,y_bin,x1,x2,x3,opinion,op,Y,time,treated,did,_diff
count,70.000000,70.00000,7.000000e+01,70.000000,70.000000,70.000000,70.000000,70.000000,70.00000,70.000000,70.000000,70.000000,70.000000,70.000000
mean,4.000000,1994.50000,1.845072e+09,0.800000,0.648001,0.133869,0.761851,2.442857,0.50000,1.845072,0.600000,0.428571,0.257143,0.257143
std,2.014441,2.89302,3.015167e+09,0.402888,0.468070,1.371099,1.446332,1.137448,0.50361,3.015167,0.493435,0.498445,0.440215,0.440215
min,1.000000,1990.00000,-7.863483e+09,0.000000,-0.567575,-1.621761,-1.165387,1.000000,0.00000,-7.863483,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1992.00000,2.465716e+08,1.000000,0.328960,-1.215649,-0.079315,1.000000,0.00000,0.246572,0.000000,0.000000,0.000000,0.000000
50%,4.000000,1994.50000,1.897898e+09,1.000000,0.641262,-0.462143,0.514191,2.500000,0.50000,1.897898,1.000000,0.000000,0.000000,0.000000
75%,6.000000,1997.00000,3.372262e+09,1.000000,1.095843,1.607754,1.154864,3.000000,1.00000,3.372262,1.000000,1.000000,0.750000,0.750000
max,7.000000,1999.00000,8.941232e+09,1.000000,1.446412,2.530277,7.168922,4.000000,1.00000,8.941232,1.000000,1.000000,1.000000,1.000000


In [6]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Parto de tu df original
df_reg = df.copy()

# ==== Generar las variables como en Stata ====

# Y = y / 1e9
df_reg["Y"] = df_reg["y"] / 1_000_000_000

# time = (year >= 1994) & !missing(year)
df_reg["time"] = ((df_reg["year"] >= 1994) & df_reg["year"].notna()).astype(int)

# treated = (country > 4) & !missing(country)
df_reg["treated"] = ((df_reg["country"] > 4) & df_reg["country"].notna()).astype(int)

# did = time * treated
df_reg["did"] = df_reg["time"] * df_reg["treated"]

# ==== Misma muestra que Stata: sin missing en las variables del modelo ====
vars_modelo = ["Y", "time", "treated", "did"]
df_reg = df_reg.dropna(subset=vars_modelo)

# ==== Regresión OLS: Y time treated did ====
X = df_reg[["time", "treated", "did"]]
X = sm.add_constant(X)  # agrega _cons
y = df_reg["Y"]

model = sm.OLS(y, X)
res = model.fit()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     1.984
Date:                Fri, 14 Nov 2025   Prob (F-statistic):              0.125
Time:                        18:43:26   Log-Likelihood:                -173.06
No. Observations:                  70   AIC:                             354.1
Df Residuals:                      66   BIC:                             363.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3581      0.738      0.485      0.6

In [10]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

# ==========================
# 1) Partimos de tu df
# ==========================
df_fe = df.copy()

# Y = y / 1e9  (como en Stata: gen Y = y/1000000000)
df_fe["Y"] = df_fe["y"] / 1_000_000_000

# time = (year >= 1994) & !missing(year)
df_fe["time"] = ((df_fe["year"] >= 1994) & df_fe["year"].notna()).astype(int)

# treated = (country > 4) & !missing(country)
df_fe["treated"] = ((df_fe["country"] > 4) & df_fe["country"].notna()).astype(int)

# did = time * treated
df_fe["did"] = df_fe["time"] * df_fe["treated"]

# ==========================
# 2) Definimos el panel
# ==========================
# xtset country year  <->  índice multinivel (country, year)
df_fe = df_fe.set_index(["country", "year"])

# ==========================
# 3) Modelo FE (within) tipo: xtreg ..., fe
# ==========================
y = df_fe["Y"]

# Incluimos time, treated y did como en Stata + constante
# (treated será omitida por colinealidad con los efectos fijos)
X = df_fe[["time", "treated", "did"]]
X = sm.add_constant(X)

# entity_effects=True  -> efectos fijos por país (country)
mod_fe = PanelOLS(y, X, entity_effects=True, drop_absorbed=True)

# Para replicar Stata, usá var/cov clásica (no robusta):
res_fe = mod_fe.fit(cov_type="unadjusted")

print(res_fe.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                      Y   R-squared:                        0.0968
Estimator:                   PanelOLS   R-squared (Between):             -0.5101
No. Observations:                  70   R-squared (Within):               0.0968
Date:                Fri, Nov 14 2025   R-squared (Overall):             -0.0035
Time:                        18:59:03   Log-likelihood                   -166.19
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3.2697
Entities:                           7   P-value                           0.0448
Avg Obs:                      10.0000   Distribution:                    F(2,61)
Min Obs:                      10.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             3.2697
                            

C:\Users\HP\AppData\Local\Temp\ipykernel_32968\1368542021.py:43: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

treated

  res_fe = mod_fe.fit(cov_type="unadjusted")


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

def did_diff_table(df, yvar="Y", treated="treated", time="time"):
    df = df.copy()

    # ===============================
    # A. Medias por grupo y período
    # ===============================
    mean_before_control  = df[(df[treated]==0) & (df[time]==0)][yvar].mean()
    mean_before_treated  = df[(df[treated]==1) & (df[time]==0)][yvar].mean()
    diff_before = mean_before_treated - mean_before_control

    mean_after_control   = df[(df[treated]==0) & (df[time]==1)][yvar].mean()
    mean_after_treated   = df[(df[treated]==1) & (df[time]==1)][yvar].mean()
    diff_after = mean_after_treated - mean_after_control

    # ===============================
    # B. Regresión: Y = b0 + time + treated + did
    # ===============================
    df["did"] = df[treated] * df[time]

    X = df[[time, treated, "did"]]
    X = sm.add_constant(X)
    y = df[yvar]

    model = sm.OLS(y, X).fit()

    b_time, b_treated, b_did = model.params[time], model.params[treated], model.params["did"]
    se_time, se_treated, se_did = model.bse[time], model.bse[treated], model.bse["did"]

    # ===============================
    # C. Errores std de las "diff"
    # ===============================
    # before diff = treated - control  = β_treated
    se_before = model.bse["treated"]
    t_before = model.tvalues["treated"]
    p_before = model.pvalues["treated"]

    # after diff  = (β_treated + β_did)
    diff_after_coef = b_treated + b_did
    # Var(a+b) = Var(a)+Var(b)+2Cov(a,b)
    cov = model.cov_params().loc["treated","did"]
    se_after = np.sqrt(model.bse["treated"]**2 + model.bse["did"]**2 + 2*cov)
    t_after = diff_after_coef / se_after
    df_resid = int(model.df_resid)
    p_after = 2*(1-stats.t.cdf(abs(t_after), df_resid))

    # DID effect = β_did
    did_coef = b_did
    did_se = se_did
    did_t = model.tvalues["did"]
    did_p = model.pvalues["did"]

    # ===============================
    # D. Construcción de tabla
    # ===============================
    tabla = f"""
DIFFERENCE-IN-DIFFERENCES ESTIMATION RESULTS
Number of observations in the DIFF-IN-DIFF: {len(df)}

            Before       After
Control:    {len(df[(df[treated]==0) & (df[time]==0)])}     {len(df[(df[treated]==0) & (df[time]==1)])}     {len(df[df[treated]==0])}
Treated:    {len(df[(df[treated]==1) & (df[time]==0)])}     {len(df[(df[treated]==1) & (df[time]==1)])}     {len(df[df[treated]==1])}

--------------------------------------------------------
Outcome var.   | {yvar}
----------------+---------+---------+---------+---------
Before
 Control        | {mean_before_control:.3f}
 Treated        | {mean_before_treated:.3f}
 Diff (T-C)     | {diff_before:.3f}   | {se_before:.3f} | {t_before:.2f} | {p_before:.3f}

After
 Control        | {mean_after_control:.3f}
 Treated        | {mean_after_treated:.3f}
 Diff (T-C)     | {diff_after:.3f}   | {se_after:.3f} | {t_after:.2f} | {p_after:.3f}

Diff-in-Diff    | {did_coef:.3f}   | {did_se:.3f} | {did_t:.2f} | {did_p:.3f}
--------------------------------------------------------
R-square: {model.rsquared:.2f}
"""

    print(tabla)
    return model

In [12]:
res = did_diff_table(df_reg, yvar="Y", treated="treated", time="time")


DIFFERENCE-IN-DIFFERENCES ESTIMATION RESULTS
Number of observations in the DIFF-IN-DIFF: 70

            Before       After
Control:    16     24     40
Treated:    12     18     30

--------------------------------------------------------
Outcome var.   | Y
----------------+---------+---------+---------+---------
Before
 Control        | 0.358
 Treated        | 2.134
 Diff (T-C)     | 1.776   | 1.128 | 1.58 | 0.120

After
 Control        | 2.648
 Treated        | 1.904
 Diff (T-C)     | -0.744   | 0.921 | -0.81 | 0.422

Diff-in-Diff    | -2.520   | 1.456 | -1.73 | 0.088
--------------------------------------------------------
R-square: 0.08



In [13]:
import pandas as pd
import pyreadstat as st
pd.set_option('display.max_columns', None)
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 2T\[MT08-MT13] Microeconometría II\Clases prácticas\PS\PS3-20251111\CardKrueger1994.dta"

df, meta = st.read_dta(path)
df.head(1)

,id,t,treated,fte,bk,kfc,roys,wendys
0,1,0,1,31.0,1,0,0,0


In [14]:
meta.column_names_to_labels

{'id': 'Store ID',
 't': 'Feb. 1992 = 0; Nov. 1992 = 1',
 'treated': 'New Jersey = 1; Pennsylvania = 0',
 'fte': 'Output: Full Time Employment',
 'bk': 'Burger King == 1',
 'kfc': 'Kentuky Fried Chiken == 1',
 'roys': 'Roy Rogers == 1',
 'wendys': "Wendy's == 1"}

In [15]:
df.describe()

,id,t,treated,fte,bk,kfc,roys,wendys
count,820.000000,820.000000,820.000000,801.000000,820.000000,820.000000,820.000000,820.000000
mean,246.507317,0.500000,0.807317,17.594569,0.417073,0.195122,0.241463,0.146341
std,148.141276,0.500305,0.394647,9.022517,0.493376,0.396536,0.428232,0.353664
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,119.000000,0.000000,1.000000,11.500000,0.000000,0.000000,0.000000,0.000000
50%,237.500000,0.500000,1.000000,16.500000,0.000000,0.000000,0.000000,0.000000
75%,372.000000,1.000000,1.000000,22.000000,1.000000,0.000000,0.000000,0.000000
max,522.000000,1.000000,1.000000,80.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

def did_diff_table(df, yvar="Y", treated="treated", time="time"):
    df = df.copy()

    # =====================================
    # 0) Nos quedamos sólo con lo necesario
    #    y dropeamos filas con NaN
    # =====================================
    cols = [yvar, treated, time]
    df = df[cols].dropna().copy()

    # Aseguramos que treated y time sean 0/1
    df[treated] = df[treated].astype(int)
    df[time]    = df[time].astype(int)

    # (Chequeo rápido opcional)
    # print(df[treated].value_counts())
    # print(df[time].value_counts())

    # =====================================
    # A. Medias por grupo y período
    # =====================================
    mask_bc = (df[treated] == 0) & (df[time] == 0)  # Before Control
    mask_bt = (df[treated] == 1) & (df[time] == 0)  # Before Treated
    mask_ac = (df[treated] == 0) & (df[time] == 1)  # After Control
    mask_at = (df[treated] == 1) & (df[time] == 1)  # After Treated

    mean_before_control  = df.loc[mask_bc, yvar].mean()
    mean_before_treated  = df.loc[mask_bt, yvar].mean()
    diff_before = mean_before_treated - mean_before_control

    mean_after_control   = df.loc[mask_ac, yvar].mean()
    mean_after_treated   = df.loc[mask_at, yvar].mean()
    diff_after = mean_after_treated - mean_after_control

    # =====================================
    # B. Regresión: Y = b0 + time + treated + did
    # =====================================
    df["did"] = df[treated] * df[time]

    X = df[[time, treated, "did"]]
    X = sm.add_constant(X)
    y = df[yvar]

    model = sm.OLS(y, X).fit()

    b_time    = model.params[time]
    b_treated = model.params[treated]
    b_did     = model.params["did"]

    se_time    = model.bse[time]
    se_treated = model.bse[treated]
    se_did     = model.bse["did"]

    # =====================================
    # C. Errores std de las "diff"
    # =====================================
    # Before diff (T-C) = β_treated
    se_before = se_treated
    t_before  = model.tvalues[treated]
    p_before  = model.pvalues[treated]

    # After diff (T-C) = β_treated + β_did
    diff_after_coef = b_treated + b_did
    cov_td = model.cov_params().loc[treated, "did"]
    se_after = np.sqrt(se_treated**2 + se_did**2 + 2 * cov_td)

    df_resid = int(model.df_resid)
    t_after = diff_after_coef / se_after
    p_after = 2 * (1 - stats.t.cdf(abs(t_after), df_resid))

    # DID effect = β_did
    did_coef = b_did
    did_se   = se_did
    did_t    = model.tvalues["did"]
    did_p    = model.pvalues["did"]

    # =====================================
    # D. Conteos (usamos el df ya limpio)
    # =====================================
    n_bc = mask_bc.sum()
    n_bt = mask_bt.sum()
    n_ac = mask_ac.sum()
    n_at = mask_at.sum()

    n_control = (df[treated] == 0).sum()
    n_treated = (df[treated] == 1).sum()

    # =====================================
    # E. Tabla "tipo Stata"
    # =====================================
    tabla = f"""
DIFFERENCE-IN-DIFFERENCES ESTIMATION RESULTS
Number of observations in the DIFF-IN-DIFFERENCE: {len(df)}

            Before       After
Control:    {n_bc:<3}        {n_ac:<3}        {n_control}
Treated:    {n_bt:<3}        {n_at:<3}        {n_treated}

--------------------------------------------------------
Outcome var.   | {yvar:>7} | S. Err. |  |t|   |  P>|t|
----------------+---------+---------+------+---------
Before
 Control        | {mean_before_control:7.3f}
 Treated        | {mean_before_treated:7.3f}
 Diff (T-C)     | {diff_before:7.3f} | {se_before:7.3f} | {t_before:4.2f} | {p_before:5.3f}

After
 Control        | {mean_after_control:7.3f}
 Treated        | {mean_after_treated:7.3f}
 Diff (T-C)     | {diff_after:7.3f} | {se_after:7.3f} | {t_after:4.2f} | {p_after:5.3f}

Diff-in-Diff    | {did_coef:7.3f} | {did_se:7.3f} | {did_t:4.2f} | {did_p:5.3f}
--------------------------------------------------------
R-square: {model.rsquared:6.2f}
"""

    print(tabla)
    return model

In [20]:
df_did = df.copy()
res = did_diff_table(df_did, yvar="fte", treated="treated", time="t")


DIFFERENCE-IN-DIFFERENCES ESTIMATION RESULTS
Number of observations in the DIFF-IN-DIFFERENCE: 801

            Before       After
Control:    78         77         155
Treated:    326        320        646

--------------------------------------------------------
Outcome var.   |     fte | S. Err. |  |t|   |  P>|t|
----------------+---------+---------+------+---------
Before
 Control        |  19.949
 Treated        |  17.065
 Diff (T-C)     |  -2.884 |   1.135 | -2.54 | 0.011

After
 Control        |  17.542
 Treated        |  17.573
 Diff (T-C)     |   0.030 |   1.143 | 0.03 | 0.979

Diff-in-Diff    |   2.914 |   1.611 | 1.81 | 0.071
--------------------------------------------------------
R-square:   0.01



In [22]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

def did_diff_table_bootstrap(
    df,
    yvar="Y",
    treated="treated",
    time="time",
    reps=50,
    random_state=None
):
    df = df.copy()

    # ================================
    # 0) Limpiamos y aseguramos 0/1
    # ================================
    cols = [yvar, treated, time]
    df = df[cols].dropna().copy()

    df[treated] = df[treated].astype(int)
    df[time]    = df[time].astype(int)

    # ================================
    # A. Medias por grupo y período
    # ================================
    mask_bc = (df[treated] == 0) & (df[time] == 0)  # Before Control
    mask_bt = (df[treated] == 1) & (df[time] == 0)  # Before Treated
    mask_ac = (df[treated] == 0) & (df[time] == 1)  # After Control
    mask_at = (df[treated] == 1) & (df[time] == 1)  # After Treated

    mean_before_control  = df.loc[mask_bc, yvar].mean()
    mean_before_treated  = df.loc[mask_bt, yvar].mean()
    diff_before          = mean_before_treated - mean_before_control

    mean_after_control   = df.loc[mask_ac, yvar].mean()
    mean_after_treated   = df.loc[mask_at, yvar].mean()
    diff_after           = mean_after_treated - mean_after_control

    n_bc = mask_bc.sum()
    n_bt = mask_bt.sum()
    n_ac = mask_ac.sum()
    n_at = mask_at.sum()

    n_control = (df[treated] == 0).sum()
    n_treated = (df[treated] == 1).sum()

    # ================================
    # B. Regresión base (punto estimado)
    # ================================
    df["did"] = df[treated] * df[time]

    X = df[[time, treated, "did"]]
    X = sm.add_constant(X)
    y = df[yvar]

    model = sm.OLS(y, X).fit()

    b_treated = model.params[treated]
    b_did     = model.params["did"]

    # Punto estimado de cada contraste (como en diff sin bootstrap)
    diff_before_hat = b_treated
    diff_after_hat  = b_treated + b_did
    did_hat         = b_did

    df_resid = int(model.df_resid)

    # ================================
    # C. Bootstrap de regresión
    # ================================
    rng = np.random.default_rng(random_state)

    boot_diff_before = []
    boot_diff_after  = []
    boot_did         = []

    n = len(df)

    for _ in range(reps):
        # Muestra bootstrap (re-muestreo de filas con reemplazo)
        sample_idx = rng.integers(0, n, size=n)
        df_b = df.iloc[sample_idx].copy()

        Xb = df_b[[time, treated, "did"]]
        Xb = sm.add_constant(Xb)
        yb = df_b[yvar]

        try:
            mb = sm.OLS(yb, Xb).fit()

            b_treated_b = mb.params[treated]
            b_did_b     = mb.params["did"]

            boot_diff_before.append(b_treated_b)
            boot_diff_after.append(b_treated_b + b_did_b)
            boot_did.append(b_did_b)
        except Exception:
            # Si alguna réplica es degenerada, la salteamos
            continue

    boot_diff_before = np.array(boot_diff_before)
    boot_diff_after  = np.array(boot_diff_after)
    boot_did         = np.array(boot_did)

    # SE bootstrap (ddof=1)
    se_before_bs = boot_diff_before.std(ddof=1)
    se_after_bs  = boot_diff_after.std(ddof=1)
    se_did_bs    = boot_did.std(ddof=1)

    # ================================
    # D. Estadísticos t y p (usando df_resid original)
    # ================================
    t_before = diff_before_hat / se_before_bs
    p_before = 2 * (1 - stats.t.cdf(abs(t_before), df_resid))

    t_after = diff_after_hat / se_after_bs
    p_after = 2 * (1 - stats.t.cdf(abs(t_after), df_resid))

    t_did = did_hat / se_did_bs
    p_did = 2 * (1 - stats.t.cdf(abs(t_did), df_resid))

    # ================================
    # E. Tabla estilo Stata
    # ================================
    tabla = f"""
DIFFERENCE-IN-DIFFERENCES ESTIMATION RESULTS
Number of observations in the DIFF-IN-DIFFERENCE: {len(df)}

            Before         After
   Control: {n_bc:<4}          {n_ac:<4}        {n_control}
   Treated: {n_bt:<4}          {n_at:<4}        {n_treated}
            {n_bc+n_bt:<4}          {n_ac+n_at:<4}

Bootstrapped Standard Errors ({reps} replications)

--------------------------------------------------------
 Outcome var.   | {yvar:>7} | S. Err. |   |t|   |  P>|t|
----------------+---------+---------+---------+---------
Before
   Control      | {mean_before_control:7.3f} |         |         |      
   Treated      | {mean_before_treated:7.3f} |         |         |      
   Diff (T-C)   | {diff_before_hat:7.3f} | {se_before_bs:7.3f} | {t_before:7.2f} | {p_before:7.3f}
After
   Control      | {mean_after_control:7.3f} |         |         |      
   Treated      | {mean_after_treated:7.3f} |         |         |      
   Diff (T-C)   | {diff_after_hat:7.3f} | {se_after_bs:7.3f} | {t_after:7.2f} | {p_after:7.3f}
                |         |         |         |      
Diff-in-Diff    | {did_hat:7.3f} | {se_did_bs:7.3f} | {t_did:7.2f} | {p_did:7.3f}
--------------------------------------------------------
R-square:    {model.rsquared:4.2f}
* Means and Standard Errors are estimated by linear regression
**Inference: *** p<0.01; ** p<0.05; * p<0.1
"""
    print(tabla)
    return model, {
        "diff_before": (diff_before_hat, se_before_bs, t_before, p_before),
        "diff_after":  (diff_after_hat,  se_after_bs,  t_after,  p_after),
        "did":         (did_hat,         se_did_bs,    t_did,    p_did),
    }


In [28]:
df_did_b = df.copy()

model, stats_did = did_diff_table_bootstrap(
    df_did_b,
    yvar="fte",
    treated="treated",
    time="t",
    reps=50,            # igual que Stata
    random_state=5  # opcional, para reproducibilidad
)


DIFFERENCE-IN-DIFFERENCES ESTIMATION RESULTS
Number of observations in the DIFF-IN-DIFFERENCE: 801

            Before         After
   Control: 78            77          155
   Treated: 326           320         646
            404           397 

Bootstrapped Standard Errors (50 replications)

--------------------------------------------------------
 Outcome var.   |     fte | S. Err. |   |t|   |  P>|t|
----------------+---------+---------+---------+---------
Before
   Control      |  19.949 |         |         |      
   Treated      |  17.065 |         |         |      
   Diff (T-C)   |  -2.884 |   1.301 |   -2.22 |   0.027
After
   Control      |  17.542 |         |         |      
   Treated      |  17.573 |         |         |      
   Diff (T-C)   |   0.030 |   1.042 |    0.03 |   0.977
                |         |         |         |      
Diff-in-Diff    |   2.914 |   1.691 |    1.72 |   0.085
--------------------------------------------------------
R-square:    0.01
* Means

In [37]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

def did_diff_table_covariates(
    df,
    yvar="fte",
    treated="treated",
    time="t",
    covariates=None
):
    if covariates is None or len(covariates) == 0:
        raise ValueError("Debes pasar una lista de covariables en 'covariates'.")

    df = df.copy()

    # 1) Limpiamos y aseguramos tipos
    cols = [yvar, treated, time] + list(covariates)
    df = df[cols].dropna().copy()

    df[treated] = df[treated].astype(int)
    df[time]    = df[time].astype(int)

    # Máscaras de celdas
    mask_bc = (df[treated] == 0) & (df[time] == 0)  # Before Control
    mask_bt = (df[treated] == 1) & (df[time] == 0)  # Before Treated
    mask_ac = (df[treated] == 0) & (df[time] == 1)  # After Control
    mask_at = (df[treated] == 1) & (df[time] == 1)  # After Treated

    n_bc = mask_bc.sum()
    n_bt = mask_bt.sum()
    n_ac = mask_ac.sum()
    n_at = mask_at.sum()

    n_control = (df[treated] == 0).sum()
    n_treated = (df[treated] == 1).sum()

    # 2) Crear did y correr regresión con covariables
    df["did"] = df[treated] * df[time]

    X = df[[time, treated, "did"] + list(covariates)]
    X = sm.add_constant(X)
    y = df[yvar]

    model   = sm.OLS(y, X).fit()
    params  = model.params
    covb    = model.cov_params()
    df_resid = int(model.df_resid)

    # Medias GLOBALES de las covariables (sobre toda la muestra)
    mean_covs_global = df[covariates].mean()

    # Helper: construye el vector a para cada celda usando medias GLOBALES
    def build_a(t_val, d_val):
        a = pd.Series(0.0, index=params.index)
        a["const"]  = 1.0
        a[time]     = float(t_val)
        a[treated]  = float(d_val)
        a["did"]    = float(t_val * d_val)
        # Usar medias GLOBALES de covariables
        for c in covariates:
            a[c] = mean_covs_global[c]
        return a

    # 3) Vectores a_cell
    a_CB = build_a(t_val=0, d_val=0)  # Control Before
    a_TB = build_a(t_val=0, d_val=1)  # Treated Before
    a_CA = build_a(t_val=1, d_val=0)  # Control After
    a_TA = build_a(t_val=1, d_val=1)  # Treated After

    # Medias AJUSTADAS
    mean_CB = float(a_CB @ params)
    mean_TB = float(a_TB @ params)
    mean_CA = float(a_CA @ params)
    mean_TA = float(a_TA @ params)

    # 4) Contrastes como combinaciones lineales de β
    # Before: Treated - Control
    c_before   = a_TB - a_CB
    diff_before = float(c_before @ params)
    var_before  = float(c_before @ covb @ c_before)
    se_before   = np.sqrt(var_before)

    # After: Treated - Control
    c_after   = a_TA - a_CA
    diff_after = float(c_after @ params)
    var_after  = float(c_after @ covb @ c_after)
    se_after   = np.sqrt(var_after)

    # DID = (After T-C) - (Before T-C)
    c_did   = c_after - c_before
    did_coef = float(c_did @ params)
    var_did  = float(c_did @ covb @ c_did)
    se_did   = np.sqrt(var_did)

    # t y p-values
    t_before = diff_before / se_before
    p_before = 2 * (1 - stats.t.cdf(abs(t_before), df_resid))

    t_after = diff_after / se_after
    p_after = 2 * (1 - stats.t.cdf(abs(t_after), df_resid))

    t_did = did_coef / se_did
    p_did = 2 * (1 - stats.t.cdf(abs(t_did), df_resid))

    # 5) Tabla estilo diff
    tabla = f"""
DIFFERENCE-IN-DIFFERENCES WITH COVARIATES

DIFFERENCE-IN-DIFFERENCES ESTIMATION RESULTS
Number of observations in the DIFF-IN-DIFF: {len(df)}
            Before         After    
   Control: {n_bc:<4}           {n_ac:<4}         {n_control}
   Treated: {n_bt:<4}           {n_at:<4}         {n_treated}
            {n_bc+n_bt:<4}           {n_ac+n_at:<4}
--------------------------------------------------------
 Outcome var.   | {yvar:>7} | S. Err. |   |t|   |  P>|t|
----------------+---------+---------+---------+---------
Before          |         |         |         | 
   Control      | {mean_CB:7.3f} |         |         | 
   Treated      | {mean_TB:7.3f} |         |         | 
   Diff (T-C)   | {diff_before:7.3f} | {se_before:7.3f} | {t_before:7.2f} | {p_before:7.3f}
After           |         |         |         | 
   Control      | {mean_CA:7.3f} |         |         | 
   Treated      | {mean_TA:7.3f} |         |         | 
   Diff (T-C)   | {diff_after:7.3f} | {se_after:7.3f} | {t_after:7.2f} | {p_after:7.3f}
                |         |         |         | 
Diff-in-Diff    | {did_coef:7.3f} | {se_did:7.3f} | {t_did:7.2f} | {p_did:7.3f}
--------------------------------------------------------
R-square:    {model.rsquared:4.2f}
* Means and Standard Errors are estimated by linear regression
**Inference: *** p<0.01; ** p<0.05; * p<0.1
"""
    print(tabla)
    return model

In [38]:
df_diff_c = df.copy()
df_diff_c = df_diff_c[df_diff_c["fte"].notna()].copy()

did_model = did_diff_table_covariates(df_diff_c, yvar="fte",
                                      treated="treated",
                                      time="t",
                                      covariates=["bk", "kfc", "roys"])


DIFFERENCE-IN-DIFFERENCES WITH COVARIATES

DIFFERENCE-IN-DIFFERENCES ESTIMATION RESULTS
Number of observations in the DIFF-IN-DIFF: 801
            Before         After    
   Control: 78             77           155
   Treated: 326            320          646
            404            397 
--------------------------------------------------------
 Outcome var.   |     fte | S. Err. |   |t|   |  P>|t|
----------------+---------+---------+---------+---------
Before          |         |         |         | 
   Control      |  19.487 |         |         | 
   Treated      |  17.163 |         |         | 
   Diff (T-C)   |  -2.324 |   1.031 |   -2.25 |   0.024
After           |         |         |         | 
   Control      |  17.084 |         |         | 
   Treated      |  17.696 |         |         | 
   Diff (T-C)   |   0.611 |   1.037 |    0.59 |   0.556
                |         |         |         | 
Diff-in-Diff    |   2.935 |   1.460 |    2.01 |   0.045
--------------------------

In [39]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

def did_diff_table_ddd(
    df,
    yvar="fte",
    treated="treated",
    time="t",
    ddd_var="bk"
):
    df = df.copy()

    # 1) Limpiamos y aseguramos tipos
    cols = [yvar, treated, time, ddd_var]
    df = df[cols].dropna().copy()

    df[treated] = df[treated].astype(int)
    df[time]    = df[time].astype(int)
    df[ddd_var] = df[ddd_var].astype(int)

    # Máscaras de las 8 celdas
    # Control (A): treated=0, ddd_var=1
    # Control (B): treated=0, ddd_var=0
    # Treated (A): treated=1, ddd_var=1
    # Treated (B): treated=1, ddd_var=0
    
    mask_b_ca = (df[treated] == 0) & (df[ddd_var] == 1) & (df[time] == 0)  # Before Control A
    mask_b_cb = (df[treated] == 0) & (df[ddd_var] == 0) & (df[time] == 0)  # Before Control B
    mask_b_ta = (df[treated] == 1) & (df[ddd_var] == 1) & (df[time] == 0)  # Before Treated A
    mask_b_tb = (df[treated] == 1) & (df[ddd_var] == 0) & (df[time] == 0)  # Before Treated B
    
    mask_a_ca = (df[treated] == 0) & (df[ddd_var] == 1) & (df[time] == 1)  # After Control A
    mask_a_cb = (df[treated] == 0) & (df[ddd_var] == 0) & (df[time] == 1)  # After Control B
    mask_a_ta = (df[treated] == 1) & (df[ddd_var] == 1) & (df[time] == 1)  # After Treated A
    mask_a_tb = (df[treated] == 1) & (df[ddd_var] == 0) & (df[time] == 1)  # After Treated B

    n_b_ca = mask_b_ca.sum()
    n_b_cb = mask_b_cb.sum()
    n_b_ta = mask_b_ta.sum()
    n_b_tb = mask_b_tb.sum()
    n_a_ca = mask_a_ca.sum()
    n_a_cb = mask_a_cb.sum()
    n_a_ta = mask_a_ta.sum()
    n_a_tb = mask_a_tb.sum()

    n_ca = n_b_ca + n_a_ca
    n_cb = n_b_cb + n_a_cb
    n_ta = n_b_ta + n_a_ta
    n_tb = n_b_tb + n_a_tb

    # 2) Crear interacciones y correr regresión
    df["did"] = df[treated] * df[time]
    df["t_ddd"] = df[time] * df[ddd_var]
    df["treated_ddd"] = df[treated] * df[ddd_var]
    df["ddd"] = df[treated] * df[time] * df[ddd_var]

    X = df[[time, treated, ddd_var, "did", "t_ddd", "treated_ddd", "ddd"]]
    X = sm.add_constant(X)
    y = df[yvar]

    model   = sm.OLS(y, X).fit()
    params  = model.params
    covb    = model.cov_params()
    df_resid = int(model.df_resid)

    # Helper: construye el vector a para cada celda
    def build_a(t_val, d_val, ddd_val):
        a = pd.Series(0.0, index=params.index)
        a["const"]  = 1.0
        a[time]     = float(t_val)
        a[treated]  = float(d_val)
        a[ddd_var]  = float(ddd_val)
        a["did"]    = float(t_val * d_val)
        a["t_ddd"]  = float(t_val * ddd_val)
        a["treated_ddd"] = float(d_val * ddd_val)
        a["ddd"]    = float(t_val * d_val * ddd_val)
        return a

    # 3) Vectores a_cell para las 8 celdas
    a_b_ca = build_a(t_val=0, d_val=0, ddd_val=1)  # Before Control A
    a_b_cb = build_a(t_val=0, d_val=0, ddd_val=0)  # Before Control B
    a_b_ta = build_a(t_val=0, d_val=1, ddd_val=1)  # Before Treated A
    a_b_tb = build_a(t_val=0, d_val=1, ddd_val=0)  # Before Treated B
    
    a_a_ca = build_a(t_val=1, d_val=0, ddd_val=1)  # After Control A
    a_a_cb = build_a(t_val=1, d_val=0, ddd_val=0)  # After Control B
    a_a_ta = build_a(t_val=1, d_val=1, ddd_val=1)  # After Treated A
    a_a_tb = build_a(t_val=1, d_val=1, ddd_val=0)  # After Treated B

    # Medias predichas
    mean_b_ca = float(a_b_ca @ params)
    mean_b_cb = float(a_b_cb @ params)
    mean_b_ta = float(a_b_ta @ params)
    mean_b_tb = float(a_b_tb @ params)
    mean_a_ca = float(a_a_ca @ params)
    mean_a_cb = float(a_a_cb @ params)
    mean_a_ta = float(a_a_ta @ params)
    mean_a_tb = float(a_a_tb @ params)

    # 4) Contrastes
    # Before: [(Treated A - Control A) - (Treated B - Control B)]
    c_before = (a_b_ta - a_b_ca) - (a_b_tb - a_b_cb)
    diff_before = float(c_before @ params)
    var_before  = float(c_before @ covb @ c_before)
    se_before   = np.sqrt(var_before)

    # After: [(Treated A - Control A) - (Treated B - Control B)]
    c_after = (a_a_ta - a_a_ca) - (a_a_tb - a_a_cb)
    diff_after = float(c_after @ params)
    var_after  = float(c_after @ covb @ c_after)
    se_after   = np.sqrt(var_after)

    # DDD = After - Before
    c_ddd = c_after - c_before
    ddd_coef = float(c_ddd @ params)
    var_ddd  = float(c_ddd @ covb @ c_ddd)
    se_ddd   = np.sqrt(var_ddd)

    # t y p-values
    t_before = diff_before / se_before
    p_before = 2 * (1 - stats.t.cdf(abs(t_before), df_resid))

    t_after = diff_after / se_after
    p_after = 2 * (1 - stats.t.cdf(abs(t_after), df_resid))

    t_ddd = ddd_coef / se_ddd
    p_ddd = 2 * (1 - stats.t.cdf(abs(t_ddd), df_resid))

    # 5) Tabla estilo diff
    tabla = f"""
TRIPLE DIFFERENCE (DDD) ESTIMATION RESULTS
Notation of DDD:
   Control (A)     {treated} = 0 and {ddd_var} = 1
   Control (B)     {treated} = 0 and {ddd_var} = 0
   Treated (A)     {treated} = 1 and {ddd_var} = 1
   Treated (B)     {treated} = 1 and {ddd_var} = 0
Number of observations in the DDD: {len(df)}
               Before      After    
   Control (A):{n_b_ca:<4}        {n_a_ca:<4}        {n_ca}
   Control (B):{n_b_cb:<4}        {n_a_cb:<4}        {n_cb}
   Treated (A):{n_b_ta:<4}        {n_a_ta:<4}        {n_ta}
   Treated (B):{n_b_tb:<4}        {n_a_tb:<4}        {n_tb}
               {n_b_ca+n_b_cb+n_b_ta+n_b_tb:<4}       {n_a_ca+n_a_cb+n_a_ta+n_a_tb:<4}
--------------------------------------------------------
 Outcome var.   | {yvar:>7} | S. Err. |   |t|   |  P>|t|
----------------+---------+---------+---------+---------
Before          |         |         |         | 
   Control (A)  | {mean_b_ca:7.3f} |         |         | 
   Control (B)  | {mean_b_cb:7.3f} |         |         | 
   Treated (A)  | {mean_b_ta:7.3f} |         |         | 
   Treated (B)  | {mean_b_tb:7.3f} |         |         | 
   Diff (T-C)   | {diff_before:7.3f} | {se_before:7.3f} | {t_before:7.2f} | {p_before:7.3f}
After           |         |         |         | 
   Control (A)  | {mean_a_ca:7.3f} |         |         | 
   Control (B)  | {mean_a_cb:7.3f} |         |         | 
   Treated (A)  | {mean_a_ta:7.3f} |         |         | 
   Treated (B)  | {mean_a_tb:7.3f} |         |         | 
   Diff (T-C)   | {diff_after:7.3f} | {se_after:7.3f} | {t_after:7.2f} | {p_after:7.3f}
                |         |         |         | 
DDD             | {ddd_coef:7.3f} | {se_ddd:7.3f} | {t_ddd:7.2f} | {p_ddd:7.3f}
--------------------------------------------------------
R-square:    {model.rsquared:4.2f}
* Means and Standard Errors are estimated by linear regression
**Inference: *** p<0.01; ** p<0.05; * p<0.1
"""
    print(tabla)
    return model

In [40]:
df_diff_dd = df.copy()

did_model = did_diff_table_ddd(
    df_diff_dd,
    yvar="fte",
    treated="treated",
    time="t",
    ddd_var="bk"
)


TRIPLE DIFFERENCE (DDD) ESTIMATION RESULTS
Notation of DDD:
   Control (A)     treated = 0 and bk = 1
   Control (B)     treated = 0 and bk = 0
   Treated (A)     treated = 1 and bk = 1
   Treated (B)     treated = 1 and bk = 0
Number of observations in the DDD: 801
               Before      After    
   Control (A):34          35          69
   Control (B):44          42          86
   Treated (A):133         132         265
   Treated (B):193         188         381
               404        397 
--------------------------------------------------------
 Outcome var.   |     fte | S. Err. |   |t|   |  P>|t|
----------------+---------+---------+---------+---------
Before          |         |         |         | 
   Control (A)  |  25.654 |         |         | 
   Control (B)  |  15.540 |         |         | 
   Treated (A)  |  18.547 |         |         | 
   Treated (B)  |  16.044 |         |         | 
   Diff (T-C)   |  -7.612 |   2.206 |   -3.45 |   0.001
After           |       

In [41]:
import pandas as pd
import pyreadstat as st
pd.set_option('display.max_columns', None)
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 2T\[MT08-MT13] Microeconometría II\Clases prácticas\PS\PS3-20251111\hospdd.dta"

df, meta = st.read_dta(path)
df.head(1)

,hospital,frequency,month,procedure,satis
0,1,3,7,1,4.106527


In [46]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

def didregress_python(df, outcome, treatment, group, time):
    """
    Replica didregress de Stata con efectos fijos de grupo y tiempo
    
    Parameters:
    -----------
    df : DataFrame
    outcome : str - variable de resultado (ej: 'satis')
    treatment : str - variable de tratamiento (ej: 'procedure')
    group : str - variable de grupo (ej: 'hospital')
    time : str - variable de tiempo (ej: 'month')
    """
    df = df.copy()
    
    # Eliminar missings
    df = df[[outcome, treatment, group, time]].dropna()
    
    # Asegurar tipos correctos
    df[group] = df[group].astype(int)
    df[time] = df[time].astype(int)
    
    # Crear treated (max de procedure por hospital)
    df['treated'] = df.groupby(group)[treatment].transform('max')
    
    # Estadísticas descriptivas
    control_groups = df[df['treated'] == 0][group].nunique()
    treatment_groups = df[df['treated'] == 1][group].nunique()
    
    control_time_min = df[df['treated'] == 0][time].min()
    control_time_max = df[df['treated'] == 0][time].max()
    treatment_time_min = df[df['treated'] == 1][time].min()
    treatment_time_max = df[df['treated'] == 1][time].max()
    
    n_clusters = df[group].nunique()
    n_obs = len(df)
    
    # Crear dummies de grupo y tiempo (omitir la primera categoría)
    group_dummies = pd.get_dummies(df[group], prefix=group, drop_first=True, dtype=float)
    time_dummies = pd.get_dummies(df[time], prefix=time, drop_first=True, dtype=float)
    
    # Construir matriz X con tipos numéricos
    X = pd.concat([
        df[[treatment]].astype(float), 
        group_dummies, 
        time_dummies
    ], axis=1)
    X = sm.add_constant(X, has_constant='add')
    
    y = df[outcome].astype(float)
    
    # Estimar con errores robustos clustered
    model = sm.OLS(y, X).fit(cov_type='cluster', 
                              cov_kwds={'groups': df[group].values})
    
    # Extraer coeficiente del tratamiento
    atet = model.params[treatment]
    se_atet = model.bse[treatment]
    t_atet = model.tvalues[treatment]
    p_atet = model.pvalues[treatment]
    ci_lower = model.conf_int().loc[treatment, 0]
    ci_upper = model.conf_int().loc[treatment, 1]
    
    # Imprimir resultados al estilo Stata
    print("Number of groups and treatment time")
    print(f"Time variable: {time}")
    print(f"Control:       {treatment} = 0")
    print(f"Treatment:     {treatment} = 1")
    print("-----------------------------------")
    print("             |   Control  Treatment")
    print("-------------+---------------------")
    print("Group        |")
    print(f"    {group:8} | {control_groups:8}  {treatment_groups:8}")
    print("-------------+---------------------")
    print("Time         |")
    print(f"     Minimum | {control_time_min:8}  {treatment_time_min:8}")
    print(f"     Maximum | {control_time_max:8}  {treatment_time_max:8}")
    print("-----------------------------------")
    print(f"Difference-in-differences regression                     Number of obs = {n_obs:,}")
    print("Data type: Repeated cross-sectional")
    print(f"                               (Std. err. adjusted for {n_clusters} clusters in {group})")
    print("-------------------------------------------------------------------------------")
    print("              |               Robust")
    print(f"        {outcome:5} | Coefficient  std. err.      t    P>|t|     [95% conf. interval]")
    print("--------------+----------------------------------------------------------------")
    print("ATET          |")
    print(f"    {treatment:9} |")
    print(f"(New vs Old)  | {atet:10.7f}  {se_atet:9.7f}  {t_atet:6.2f}   {p_atet:.3f}     {ci_lower:9.7f}  {ci_upper:9.7f}")
    print("-------------------------------------------------------------------------------")
    print("Note: ATET estimate adjusted for covariates, group effects, and time effects.")
    
    return model


# Código para ejecutar:
model = didregress_python(df, outcome='satis', treatment='procedure', 
                          group='hospital', time='month')

Number of groups and treatment time
Time variable: month
Control:       procedure = 0
Treatment:     procedure = 1
-----------------------------------
             |   Control  Treatment
-------------+---------------------
Group        |
    hospital |       28        18
-------------+---------------------
Time         |
     Minimum |        1         1
     Maximum |        7         7
-----------------------------------
Difference-in-differences regression                     Number of obs = 7,368
Data type: Repeated cross-sectional
                               (Std. err. adjusted for 46 clusters in hospital)
-------------------------------------------------------------------------------
              |               Robust
        satis | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
--------------+----------------------------------------------------------------
ATET          |
    procedure |
(New vs Old)  |  0.8479879  0.0321121   26.41   0.000     0.7850493  